In [ ]:
import sqlite3

# Create a connection to the SQLite database
conn = sqlite3.connect(':memory:')  # Use an in-memory database for demonstration
cursor = conn.cursor()

# Create tables
cursor.execute('''
CREATE TABLE student (
    snum INTEGER PRIMARY KEY,
    sname TEXT,
    major TEXT,
    lvl TEXT,
    age INTEGER
)''')

cursor.execute('''
CREATE TABLE class (
    cname TEXT PRIMARY KEY,
    meets_at TEXT,
    room TEXT,
    fid INTEGER
)''')

cursor.execute('''
CREATE TABLE enrolled (
    snum INTEGER,
    cname TEXT,
    PRIMARY KEY (snum, cname),
    FOREIGN KEY (snum) REFERENCES student(snum),
    FOREIGN KEY (cname) REFERENCES class(cname)
)''')

cursor.execute('''
CREATE TABLE faculty (
    fid INTEGER PRIMARY KEY,
    fname TEXT,
    depid INTEGER
)''')

# Insert sample data
cursor.executemany('''
INSERT INTO student (snum, sname, major, lvl, age) VALUES (?, ?, ?, ?, ?)''', [
    (1, 'Alice', 'CS', 'JR', 20),
    (2, 'Bob', 'Math', 'SR', 21),
    (3, 'Charlie', 'CS', 'JR', 19),
    (4, 'David', 'Bio', 'FR', 18),
    (5, 'Eve', 'CS', 'JR', 22)
])

cursor.executemany('''
INSERT INTO class (cname, meets_at, room, fid) VALUES (?, ?, ?, ?)''', [
    ('CS101', '10:00', 'R218', 1),
    ('CS102', '10:00', 'R219', 1),
    ('Math101', '11:00', 'R220', 2),
    ('Bio101', '10:00', 'R218', 3),
    ('CS103', '10:00', 'R218', 1)
])

cursor.executemany('''
INSERT INTO enrolled (snum, cname) VALUES (?, ?)''', [
    (1, 'CS101'),
    (1, 'CS102'),
    (3, 'CS101'),
    (5, 'CS101'),
    (2, 'Math101'),
    (4, 'Bio101'),
    (1, 'CS103'),
    (3, 'CS102')
])

cursor.executemany('''
INSERT INTO faculty (fid, fname, depid) VALUES (?, ?, ?)''', [
    (1, 'Prof. Smith', 1),
    (2, 'Prof. Johnson', 2),
    (3, 'Prof. Lee', 1)
])

# Query 1: Find the names of all juniors (level = JR) who are enrolled in a class taught by a faculty member.
query1 = '''
SELECT DISTINCT s.sname
FROM student s
JOIN enrolled e ON s.snum = e.snum
JOIN class c ON e.cname = c.cname
WHERE s.lvl = 'JR' AND c.fid IS NOT NULL;
'''
cursor.execute(query1)
juniors = cursor.fetchall()
print("Juniors enrolled in classes taught by faculty:")
print([name[0] for name in juniors])

# Query 2: Find the names of all classes that either meet in room R218 or have five or more students enrolled.
query2 = '''
SELECT DISTINCT c.cname
FROM class c
LEFT JOIN enrolled e ON c.cname = e.cname
GROUP BY c.cname, c.room
HAVING c.room = 'R218' OR COUNT(e.snum) >= 5;
'''
cursor.execute(query2)
classes = cursor.fetchall()
print("Classes meeting in R218 or with five or more students enrolled:")
print([cls[0] for cls in classes])

# Query 3: Find the names of all students who are enrolled in two classes that meet at the same time.
query3 = '''
SELECT DISTINCT s.sname
FROM student s
JOIN enrolled e1 ON s.snum = e1.snum
JOIN class c1 ON e1.cname = c1.cname
JOIN enrolled e2 ON s.snum = e2.snum
JOIN class c2 ON e2.cname = c2.cname
WHERE c1.meets_at = c2.meets_at AND c1.cname <> c2.cname;
'''
cursor.execute(query3)
students_same_time = cursor.fetchall()
print("Students enrolled in two classes that meet at the same time:")
print([name[0] for name in students_same_time])
